In [1]:

from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import pipeline
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM





In [ ]:
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,
                                          use_auth_token=True)

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True
                                             )


In [ ]:
pipe = pipeline("text-generation",
                model = model,
                tokenizer = tokenizer,
                torch_dtype = torch.bfloat16,
                device_map = "auto",
                max_new_tokens = 512,
                do_sample = True,
                top_k = 30,
                num_return_sequences = 1,
                eos_token_id = tokenizer.eos_token_id
                )


In [3]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template
    

In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, 
                          model_kwargs = {'temperature':0})


In [ ]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to French:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [11]:
instruction = "Context:\n\n{context} \n\nQuestion: {question}"
system_prompt = """
You are a biomedical researcher. For answering the question at the end, you need to first read the Context provided and then answer the Question as "True" or "False". If you don't know the answer, report as "Don't know". Give your answer in the following format.
{{
answer : <True> or <False> or <Don't know>
}}
"""
template = get_prompt(instruction, system_prompt)
print(template)


[INST]<<SYS>>

You are a biomedical researcher. For answering the question at the end, you need to first read the Context provided and then answer the Question as "True" or "False". If you don't know the answer, report as "Don't know". Give your answer in the following format.
{{
answer : <True> or <False> or <Don't know>
}}

<</SYS>>

Context:

{context} 

Question: {question}[/INST]


In [ ]:
question = "how are you today?"
output = llm_chain.run(text)

parse_text(output)


In [2]:
# MODEL_NAME = "Yukang/Llama-2-13b-chat-longlora-32k-sft"

# llm = HuggingFacePipeline.from_model_id(
#     model_id=MODEL_NAME,
#     task="text-generation",
#     model_kwargs={"temperature": 0, "max_length": 64},
# )

